# Skin Cancer Classification Challenge

**Summary : ** skin cancer.

## Preliminaries and Imports

In [1]:
#general & system\n",
import os

#data processing
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['figure.figsize'] = 10, 10  #default setting
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

#data augmentation
from PIL import Image
from random import choice
import cv2
import numpy as np
#import keras.preprocessing.image as prep

#ML part
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input
from keras.preprocessing import image as kimage
from keras.models import Sequential
from keras.layers import GlobalMaxPooling2D, Dense, BatchNormalization, GlobalAveragePooling2D,Dropout,Activation,Flatten
from keras.models import Model
from keras.optimizers import SGD
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator

#validation
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_curve, auc,classification_report,roc_auc_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_curve, auc,classification_report,roc_auc_score


from sklearn.metrics import precision_recall_curve
from imblearn.over_sampling import SMOTE
from sklearn.utils import resample

%matplotlib inline

Using TensorFlow backend.


## I. Load Data

- **Rq : ** Datasets are quite light, can be fully loaded in a laptop memory with ease.

In [2]:
%%time
meta = pd.read_csv("../data/label_learn.csv", sep = ";")

CPU times: user 4 ms, sys: 4 ms, total: 8 ms
Wall time: 5.49 ms


In [3]:
meta.head()

,name,label
0,0000000.jpg,benign
1,0000002.jpg,malignant
2,0000006.jpg,benign
3,0000008.jpg,benign
4,0000009.jpg,benign


In [4]:
%%time
train_list = os.listdir("../data/resized_train/")
len(train_list) == len(meta.name)

CPU times: user 0 ns, sys: 4 ms, total: 4 ms
Wall time: 4.05 ms


In [5]:
train_list.sort() == list(meta.name).sort()

True

In [6]:
%%time
X_train = np.array([np.array(Image.open("../data/resized_train/"+fname)) for fname in meta.name])
Y_train = [1 if x=="malignant" else 0 for x in meta.label]
Y_train = np.array(Y_train)

CPU times: user 23 s, sys: 1.62 s, total: 24.6 s
Wall time: 24.6 s


In [7]:
print(X_train.shape)
print(len(Y_train))
X_train.shape[0] == len(meta.name) == len(Y_train)

(8581, 299, 299, 3)
8581


True

In [8]:
#control
print(meta.name[0:5])
print(Y_train[0:5])
print(meta.label[0:5])

0    0000000.jpg
1    0000002.jpg
2    0000006.jpg
3    0000008.jpg
4    0000009.jpg
Name: name, dtype: object
[0 1 0 0 0]
0       benign
1    malignant
2       benign
3       benign
4       benign
Name: label, dtype: object


## II. Data Preparation

- **Class weights**

In [9]:
class_weight_or = class_weight.compute_class_weight('balanced', np.unique(Y_train), Y_train)

- **Data splitting**

In [10]:
#data splitting
X_train, X_valid, Y_train, Y_valid = train_test_split(X_train, Y_train, random_state=42, train_size=0.80)

/home/dvp_tran/anaconda2/lib/python2.7/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


- **Resampling**

In [11]:
temp = sum(Y_train)
sample_size = temp*2700/(374+254)  #1372
sample_size

2325

In [12]:
def random_undersampling(X, Y,sample_size = sample_size, n_sample=1):
    """
    X,y : numpy arrays
    return :
    5 random ensemble of indices general_balenced_set:
        general_balenced_set[0] = the shuffeled indices that inssure the class balance
    """
    indices = np.array(range(len(Y)))
    positive_samples = indices[Y==1]
    #print(type(positive_samples))
    negative_samples = indices[Y==0]
    #print(type(negative_samples))
    general_balenced_set = []
    for k in range(n_sample):
        indices_ = np.random.choice(negative_samples, sample_size, replace=False)
        #print(len(indices_))
        # append positive and negative
        balenced_set = np.append(indices_, positive_samples)
        #print(len(balenced_set))
        # shuffle indices
        np.random.shuffle(balenced_set)
        #print(len(balenced_set))
        general_balenced_set.append(balenced_set)
    return general_balenced_set

In [13]:
general_balenced_set = random_undersampling(X_train,Y_train)
X_train = X_train[general_balenced_set]
Y_train = Y_train[general_balenced_set]

## III. Deep Learning with Transfer Learning

In [14]:
weights_path = '../model/checkpoints/'

if not os.path.exists(weights_path):
    os.makedirs(weights_path)
    
epochs = 500
batch_size = 50

- **Using RESNET : **

In [15]:
#load base model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(299,299,3))
base_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 150, 150, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 150, 150, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 150, 150, 64) 0           bn_conv1[0][0]                   
__________________________________________________________________________________________________
max_poolin

In [16]:
#adding top layers
    #sequentialy
add_model = Sequential()
add_model.add(GlobalAveragePooling2D(input_shape=base_model.output_shape[1:])) #Flatten/GlobalAveragePooling2D
add_model.add(Dense(1024, activation='relu'))
add_model.add(Dropout(0.25))
add_model.add(Dense(512, activation='relu'))
add_model.add(Dropout(0.25))
add_model.add(Dense(1, activation='sigmoid'))

model = Model(inputs=base_model.input, outputs=add_model(base_model.output))
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 150, 150, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 150, 150, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 150, 150, 64) 0           bn_conv1[0][0]                   
__________________________________________________________________________________________________
max_poolin

In [17]:
from keras import backend as K

def weighted_categorical_crossentropy(weights):
    """
    [w_0, w_1] : weight of 0 class and weight of 1 class
    """
    weights = K.variable(weights)

    def loss(y_true, y_pred):
        # scale predictions so that the class probas of each sample sum to 1
        #y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
        # clip to prevent NaN's and Inf's
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        # calc
        loss = y_true * K.log(y_pred) * weights[1] + (1 - y_true) * K.log(1 - y_pred) * weights[0]
        loss = -K.sum(loss, -1)
        return loss

    return loss


In [18]:
class_weight_or

array([ 0.54296381,  6.31885125])

In [19]:
loss1 = weighted_categorical_crossentropy(class_weight_or) #(np.array([0.8,4]))

In [ ]:
#freeze lower layers of the model
#for layer in model.layers[:]:
    #layer.trainable = False
for layer in model.layers[0:]:
    layer.trainable = True
    

def sensitivity(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    return true_positives / (possible_positives + K.epsilon())

def specificity(y_true, y_pred):
    true_negatives = K.sum(K.round(K.clip((1-y_true) * (1-y_pred), 0, 1)))
    possible_negatives = K.sum(K.round(K.clip(1-y_true, 0, 1)))
    return true_negatives / (possible_negatives + K.epsilon())




#model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss="binary_crossentropy", #binary_crossentropy
              metrics=[sensitivity, specificity,"accuracy"])

In [ ]:
%%time
#Lets define the image transormations that we wan

gen = ImageDataGenerator(horizontal_flip=True,
                         vertical_flip=True,
                         width_shift_range=0.2,
                         height_shift_range=0.2,
                         zoom_range=0.2,
                         rotation_range=40)

val_datagen = ImageDataGenerator()
# Here is the function that merges our two generators
# We use the exact same generator with the same random seed for both the y and angle arrays
def gen_flow_for_one_input(X1, y):
    genX1 = gen.flow(X1, y, batch_size=batch_size, seed=42)
    while True:
        X1i = genX1.next()
        yield X1i[0], X1i[1]

def val_datagen_(X1, y):
    genX1 = val_datagen.flow(X1, y, batch_size=batch_size, seed=42)
    while True:
        X1i = genX1.next()
        yield X1i[0], X1i[1]
#Finally create out generator
gen_flow_train = gen_flow_for_one_input(X_train, Y_train)
#gen_flow_val = gen_flow_for_one_input(X_valid, Y_valid)
gen_flow_val = val_datagen_(X_valid, Y_valid)

from keras.callbacks import EarlyStopping, ModelCheckpoint
epochs_to_wait_for_improve = 30
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=epochs_to_wait_for_improve)
checkpoint_callback = ModelCheckpoint(weights_path + 'BestKerasResnet50_flat_3_loss_no_weight.h5', monitor='val_loss',
                                      verbose=1, save_best_only=True, mode='min')
#fit the model
model.fit_generator(gen_flow_train, validation_data=(X_valid, Y_valid),
                    steps_per_epoch=int(np.ceil(len(X_train)/batch_size)),
                    epochs=500, verbose=1, callbacks=[early_stopping_callback, checkpoint_callback],
                    class_weight = class_weight_or)

Epoch 1/500
57/57 [==============================] - 75s 1s/step - loss: 0.5369 - sensitivity: 0.1459 - specificity: 0.8876 - acc: 0.7477 - val_loss: 0.4271 - val_sensitivity: 0.0000e+00 - val_specificity: 0.9955 - val_acc: 0.9156
Epoch 2/500
57/57 [==============================] - 51s 901ms/step - loss: 0.4069 - sensitivity: 0.1220 - specificity: 0.9862 - acc: 0.8188 - val_loss: 0.3214 - val_sensitivity: 0.0326 - val_specificity: 0.9962 - val_acc: 0.9190
Epoch 3/500
57/57 [==============================] - 51s 901ms/step - loss: 0.3463 - sensitivity: 0.2701 - specificity: 0.9780 - acc: 0.8434 - val_loss: 0.2620 - val_sensitivity: 0.1921 - val_specificity: 0.9879 - val_acc: 0.9237
Epoch 4/500
57/57 [==============================] - 51s 899ms/step - loss: 0.3020 - sensitivity: 0.4113 - specificity: 0.9715 - acc: 0.8634 - val_loss: 0.2223 - val_sensitivity: 0.2624 - val_specificity: 0.9809 - val_acc: 0.9243
Epoch 5/500
57/57 [==============================] - 51s 903ms/step - loss: 0.2

Epoch 19/500
57/57 [==============================] - 49s 863ms/step - loss: 0.2043 - sensitivity: 0.7494 - specificity: 0.9477 - acc: 0.9055 - val_loss: 0.1566 - val_sensitivity: 0.6171 - val_specificity: 0.9536 - val_acc: 0.9289
Epoch 20/500
57/57 [==============================] - 51s 901ms/step - loss: 0.2073 - sensitivity: 0.7484 - specificity: 0.9460 - acc: 0.9066 - val_loss: 0.1555 - val_sensitivity: 0.6436 - val_specificity: 0.9501 - val_acc: 0.9289
Epoch 21/500
57/57 [==============================] - 49s 863ms/step - loss: 0.2106 - sensitivity: 0.7609 - specificity: 0.9445 - acc: 0.9059 - val_loss: 0.1589 - val_sensitivity: 0.6373 - val_specificity: 0.9439 - val_acc: 0.9231
Epoch 22/500
57/57 [==============================] - 51s 898ms/step - loss: 0.2043 - sensitivity: 0.7486 - specificity: 0.9500 - acc: 0.9076 - val_loss: 0.1532 - val_sensitivity: 0.5937 - val_specificity: 0.9534 - val_acc: 0.9272
Epoch 23/500
57/57 [==============================] - 51s 900ms/step - loss:

Epoch 39/500
57/57 [==============================] - 51s 898ms/step - loss: 0.1781 - sensitivity: 0.7732 - specificity: 0.9506 - acc: 0.9147 - val_loss: 0.1361 - val_sensitivity: 0.5949 - val_specificity: 0.9676 - val_acc: 0.9394
Epoch 40/500
57/57 [==============================] - 49s 862ms/step - loss: 0.1899 - sensitivity: 0.7802 - specificity: 0.9416 - acc: 0.9065 - val_loss: 0.1375 - val_sensitivity: 0.6240 - val_specificity: 0.9631 - val_acc: 0.9388
Epoch 41/500
57/57 [==============================] - 49s 861ms/step - loss: 0.1898 - sensitivity: 0.7694 - specificity: 0.9499 - acc: 0.9126 - val_loss: 0.1365 - val_sensitivity: 0.6143 - val_specificity: 0.9637 - val_acc: 0.9388
Epoch 42/500
57/57 [==============================] - 51s 898ms/step - loss: 0.1779 - sensitivity: 0.7790 - specificity: 0.9490 - acc: 0.9157 - val_loss: 0.1357 - val_sensitivity: 0.6007 - val_specificity: 0.9637 - val_acc: 0.9371
Epoch 43/500
57/57 [==============================] - 51s 898ms/step - loss:

57/57 [==============================] - 66s 1s/step - loss: 0.1483 - sensitivity: 0.8182 - specificity: 0.9640 - acc: 0.9330 - val_loss: 0.1333 - val_sensitivity: 0.6729 - val_specificity: 0.9649 - val_acc: 0.9423
Epoch 84/500
57/57 [==============================] - 51s 887ms/step - loss: 0.1308 - sensitivity: 0.8308 - specificity: 0.9704 - acc: 0.9403 - val_loss: 0.1357 - val_sensitivity: 0.6768 - val_specificity: 0.9663 - val_acc: 0.9447
Epoch 85/500
57/57 [==============================] - 49s 861ms/step - loss: 0.1426 - sensitivity: 0.8139 - specificity: 0.9702 - acc: 0.9382 - val_loss: 0.1348 - val_sensitivity: 0.6733 - val_specificity: 0.9636 - val_acc: 0.9412
Epoch 86/500
57/57 [==============================] - 51s 897ms/step - loss: 0.1267 - sensitivity: 0.8356 - specificity: 0.9712 - acc: 0.9428 - val_loss: 0.1270 - val_sensitivity: 0.5539 - val_specificity: 0.9757 - val_acc: 0.9435
Epoch 87/500
57/57 [==============================] - 49s 862ms/step - loss: 0.1384 - sensit

## IV. Visualize  TSNE

In [ ]:
%matplotlib inline
import numpy as np
# visualize the points' distribution of train data using features built from base_model
import matplotlib.pyplot as plt
# dimension reduction
from sklearn.manifold import TSNE 
import numpy as np

def t_sne_vis(name, base_model, x_processed_images, random_state, labels,perplexity = 30):
    """
    :param name: the name of the cnn model used to build features
    :param base_model: the model obj
    :param x_processed_images: the input images for our model
    :param random_state: for fixing the results
    :param labels: 0/1 classification labels
    :return:
    the graph of image distribution based on features extracted from the model and the t-sne features
    """
    # convert data to images
    print('Converting data points to composite image')
    X_train = x_processed_images
    print('we got %d different images of shape %dx%d ' % (len(X_train), X_train.shape[1], X_train.shape[1]))
    print('build usefull features from the selected model')
    features = base_model.predict(X_train)
    x_data1 = np.asarray(features).astype('float64')
    x_data1 = x_data1.reshape((x_data1.shape[0], -1))
    # perform t-SNE embedding
    print('performing t-sne reduction')
    vis_data = TSNE(perplexity=perplexity, random_state=random_state).fit_transform(x_data1)
    # plot the result
    fig = plt.figure(figsize=(15, 15))
    vis_x = vis_data[:, 0]
    vis_y = vis_data[:, 1]
    plt.scatter(vis_x, vis_y, c=labels, cmap=plt.cm.get_cmap("winter", 2))
    plt.colorbar(ticks=range(2))
    plt.clim(0, 1)
    plt.title(name)
    plt.grid()
    plt.show()
    fig.savefig('tsne_vis_'+name+'.png')
    return vis_data


In [ ]:
model.summary()

In [ ]:
name = 'Resnet'
print('visualizing for %s' %name)
base_model_ = Model(input=model.input, output=model.get_layer('avg_pool').output)
tsne_feats = t_sne_vis(name, base_model_, X_valid, np.random.RandomState(42), Y_valid,perplexity=30)

## V. Evaluate Model

In [ ]:
model = load_model(weights_path+"BestKerasResnet50_flat_3_loss_no_weight.h5", custom_objects={'sensitivity': sensitivity,
                                                                             "specificity":specificity})
score = model.evaluate(X_valid, Y_valid, verbose=1)
print('Test loss:', score[0])
print('Test sensitivity:', score[1])
print('Test specificity:', score[2])
print('Test acc:', score[3])

In [ ]:
#compute metrics
pred = model.predict(X_valid)
average_precision = average_precision_score(Y_valid, pred)

print('Average precision-recall score: {0:0.2f}'.format(
      average_precision))

In [ ]:
precision, recall, _ = precision_recall_curve(Y_valid, pred)

plt.step(recall, precision, color='b', alpha=0.2,
         where='post')
plt.fill_between(recall, precision, step='post', alpha=0.2,
                 color='b')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.title('2-class Precision-Recall curve: AP={0:0.2f}'.format(
          average_precision))

In [ ]:
def generate_results(y_test, y_score):
    fpr, tpr, _ = roc_curve(y_test, y_score)
    roc_auc = auc(fpr, tpr)
    plt.figure()
    plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.05])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic curve')
    plt.show()
    print('AUC: %f' % roc_auc)

In [ ]:
generate_results

#compute log loss
pred = model.predict(X_valid)
log_loss(Y_valid,pred)

## VI. Retrain on full dataset